# Imports

In [ ]:
import pandas as pd
import os
import carregamento # custom python file that loads the data from the json file and exports as csv's

# Processamento dos dados

Leva cerca de 5 minutos... Gera arquivos ".csv" no diretório de caminho "./csvs/".

In [ ]:
print("Initialized data processing.")
carregamento.start()
print("Finished processing the data. Beginning the treatment process.")

# Variáveis globais

In [ ]:
# specialFiles = [empty csv, content of that empty csv]
specialFiles = ["associacaoProdutoGrupo.csv", "id_associacaoProdutoGrupo.csv"]
csvFilesPath = "./csvs/"
csvFiles = [f for f in os.listdir(csvFilesPath) if f not in specialFiles]
alteredFiles = 0

# Funções de exportar, carregar e deletar arquivos

In [ ]:
def export(df : pd.DataFrame, fileName):
    df.to_csv(csvFilesPath + fileName, index=False)


def load(fileName : str):
    return pd.read_csv(csvFilesPath + fileName)


def deleteFile(fileName : str):
    if os.path.exists(csvFilesPath + fileName):
        os.remove(csvFilesPath + fileName)

# Pipeline para agregar subtabelas nas tabelas principais

(quando possível)

No carregamento dos dados, subtabelas recebem um identificador no formato "tabelaPrincipalId", que na verdade diz respeito ao índice (linha) a que os dados da subtabela pertencem na tabela principal. Como o índice das linhas começa em 0, a função "idToIndex" garante que o menor valor do identificador na subtabela será 0.

In [ ]:
def idToIndex(df : pd.DataFrame, idCol : str):
    df[idCol] -= df[idCol].min()
    df = df.sort_values(idCol).reset_index(drop=True)


def tryAddToMainTable(df : pd.DataFrame, mainFileName : str, otherFileName : str):
    if not os.path.exists(csvFilesPath + mainFileName + ".csv"):
        return
    
    main = load(mainFileName + ".csv").reset_index()
    
    idCol = mainFileName + "Id"
    main.rename(columns={'index': idCol}, inplace=True)
    main = main.merge(df, "left", on=idCol)
    main = main.sort_values(idCol).drop(idCol, axis=1)
    
    export(main, mainFileName + ".csv")
    deleteFile(otherFileName)
    print(otherFileName + " added to main table " + mainFileName + ' and deleted from "csvs" directory.')
    print("----------------------------------------------------------------------------------------------------------------------------")


def trySubtablePipeline(fileName : str):
    global alteredFiles

    df = load(fileName)
    idCol = [c for c in df.columns if c.endswith("Id")]
    
    if not len(idCol):
        return
    
    idToIndex(df, idCol[0])
    tryAddToMainTable(df, idCol[0].removesuffix("Id"), fileName)
    alteredFiles += 1

# Tratamento de caso especial

In [ ]:
# getting the content of the csv file, putting it in the empty file and deleting the first
specialCase = load(specialFiles[1])
idToIndex(specialCase, "associacaoProdutoGrupoId")
specialCase.drop("associacaoProdutoGrupoId", axis=1, inplace=True)
export(specialCase, specialFiles[0])
deleteFile(specialFiles[1])
alteredFiles += 1

# Execução da Pipeline

In [ ]:
for fileName in csvFiles:
    trySubtablePipeline(fileName)

print("Number of alterations made: " + str(alteredFiles))

In [ ]:
print('Current number of csv files in "csvs" directory: ' + str(len(os.listdir(csvFilesPath))))

# Tratamento específico

Após simples análise manual, localizou-se alguns casos específicos que poderiam ser tratados de forma individualizada para simplificar os dados. Logo, esse tratamento foi realizado abaixo:

### idElo

In [ ]:
def idElo():
    print("Working on idElo")
    idToValues = {2228: "[1551, 1937, 2308]", 2229: "[1551, 1937, 2308]", 1699: "[1551, 1937, 2308]",
                  1700: "[1551, 1937, 2308]", 1702: "[1740, 1742]", 1703: "[1741, 1743]", 1704: "[1211, 1214, 1692, 1740, 1742, 1829]"}

    main = load("pontoCargaDescargaAPS.csv").set_index("id")
    main["idElo"] = None

    for i in idToValues.keys():
        main.at[i, "idElo"] = idToValues[i]
    
    export(main.reset_index(), "pontoCargaDescargaAPS.csv")
    print('Modified "pontoCargaDescargaAPS.csv".')

    deleteFile("idElo.csv")
    print('Deleted "idElo.csv". Use "pontoCargaDescargaAPS.csv" instead.')
    print("----------------------------------------------------------------------------------------------------------------------------")

### pesoMedioTransporteProduto

In [ ]:
def pesoMedioTransporteProduto():
    print("Working on pesoMedioTransporteProduto")
    subtable = load("pesoMedioTransporteProduto_pesosMediosTransporteProduto.csv").drop("pesosMediosTransporteProdutoId", axis=1)
    main = load("pesoMedioTransporteProduto.csv")

    print("Subtable length (rows): " + str(len(subtable)))
    print("Main table length (rows): " + str(len(main)))

    final = pd.concat([subtable, main])
    print("Merged table length (rows): " + str(len(final)))
    
    export(final, "pesoMedioTransporteProduto.csv")
    print('Modified "pesoMedioTransporteProduto.csv".')

    deleteFile("pesoMedioTransporteProduto_pesosMediosTransporteProduto.csv")
    print('Deleted "pesoMedioTransporteProduto_pesosMediosTransporteProduto.csv". Use "pesoMedioTransporteProduto.csv" instead.')
    print("----------------------------------------------------------------------------------------------------------------------------")

### regraBlendagemProduto

In [ ]:
def regraBlendagemProduto():
    print("Working on regraBlendagemProduto")
    subtable = load("regraBlendagemProduto_regrasBlendagemProduto.csv").drop("regrasBlendagemProdutoId", axis=1)
    main = load("regraBlendagemProduto.csv")

    print("Subtable length (rows): " + str(len(subtable)))
    print("Main table length (rows): " + str(len(main)))
    
    final = pd.concat([subtable, main])
    
    print("Merged table length (rows): " + str(len(final)))
    
    export(final, "regraBlendagemProduto.csv")
    print('Modified "regraBlendagemProduto.csv".')

    deleteFile("regraBlendagemProduto_regrasBlendagemProduto.csv")
    print('Deleted "regraBlendagemProduto_regrasBlendagemProduto.csv". Use "regraBlendagemProduto.csv" instead.')
    print("----------------------------------------------------------------------------------------------------------------------------")

### siteProdutivoAPS

In [ ]:
def siteProdutivoAPS():
    print("Working on siteProdutivoAPS")
    if os.path.exists(csvFilesPath + "siteProdutivoAPS.csv"):
        os.remove(csvFilesPath + "siteProdutivoAPS.csv")
        print('Deleted "siteProdutivoAPS.csv".')
    print("----------------------------------------------------------------------------------------------------------------------------")

### submodalTransporte

In [ ]:
def submodalTransporte():
    print("Working on submodalTransporte")
    idToValues = {1707 : "['GDE', 'MIS']", 1995: "['DIV', 'GDE']"}

    submodalTable = load("submodalTransporte.csv")
    submodalTable.drop([c for c in submodalTable.columns if c.startswith("0")], axis=1, inplace=True)
    
    export(submodalTable.iloc[0:len(submodalTable) - 5], "submodalTransporte.csv")
    print('Modified "submodalTransporte.csv".')

    pontoCargaTable = load("pontoCargaDescargaAPS.csv").set_index("id")

    for i in idToValues.keys():
        pontoCargaTable.at[i, "submodalTransporte"] = idToValues[i]
    
    export(pontoCargaTable.reset_index(), "pontoCargaDescargaAPS.csv")
    print('Modified "pontoCargaDescargaAPS.csv".')
    print("----------------------------------------------------------------------------------------------------------------------------")

### variacaoCapacidadeVazaoTrecho

In [ ]:
def variacaoCapacidadeVazaoTrecho():
    print("Working on variacaoCapacidadeVazaoTrecho")

    subtable = load("variacaoCapacidadeVazaoTrecho_variacoesCapacidadeVazaoTrecho.csv").drop("variacoesCapacidadeVazaoTrechoId", axis=1)
    main = load("variacaoCapacidadeVazaoTrecho.csv")
    
    print("Subtable length (rows): " + str(len(subtable)))
    print("Main table length (rows): " + str(len(main)))
    
    final = pd.concat([subtable, main])
    print("Merged table length (rows): " + str(len(final)))

    export(final, "variacaoCapacidadeVazaoTrecho.csv")
    print('Modified "variacaoCapacidadeVazaoTrecho.csv".')

    deleteFile("variacaoCapacidadeVazaoTrecho_variacoesCapacidadeVazaoTrecho.csv")
    print('Deleted "variacaoCapacidadeVazaoTrecho_variacoesCapacidadeVazaoTrecho.csv". Use "variacaoCapacidadeVazaoTrecho.csv" instead.')
    print("----------------------------------------------------------------------------------------------------------------------------")

### grandezaEnsaioMapaAnaliseAPS

In [ ]:
def grandezaEnsaioMapaAnaliseAPS():
    print("Working on grandezaEnsaioMapaAnaliseAPS")

    subtable = load("mapaAnaliseAPS.csv")
    main = load("grandezaEnsaioMapaAnaliseAPS.csv")
    
    subtable.rename(columns={"id": "idMapaAnalise"}, inplace=True)

    main = main.merge(subtable, "left", "idMapaAnalise")

    export(main, "grandezaEnsaioMapaAnaliseAPS.csv")
    print('Modified "grandezaEnsaioMapaAnaliseAPS.csv".')

    deleteFile("mapaAnaliseAPS.csv")
    print('Deleted "mapaAnaliseAPS.csv". Use "grandezaEnsaioMapaAnaliseAPS.csv" instead.')
    print("----------------------------------------------------------------------------------------------------------------------------")

# Pipeline de tratamento para arquivos específicos

In [ ]:
def specificFilesPipeline():
    idElo()
    pesoMedioTransporteProduto()
    regraBlendagemProduto()
    siteProdutivoAPS()
    submodalTransporte()
    variacaoCapacidadeVazaoTrecho()
    grandezaEnsaioMapaAnaliseAPS()

specificFilesPipeline()

In [ ]:
csvFiles = os.listdir(csvFilesPath)
print('Final number of csv files in "csvs" directory: ' + str(len(csvFiles)))
print("Informative tables: " + str(len([f for f in csvFiles if f .endswith("APS.csv")])))

# Filtro de colunas

No processo de agregar subtabelas em suas respectivas tabelas principais, colunas com nomes idênticos receberam os sufixos "_x" e "_y" para serem diferenciadas. Isso ocorreu especificamente com *merges* realizados a partir das subtabelas prefixadas com *siteProdutivo*. Logo, essas colunas foram renomeadas e também foram excluídas as colunas de identificadores de índice em tabelas principais, pois já não são úteis. 

In [ ]:
def tryRemovingReferenceCol(fileName : str):
    global alteredFiles
    df = load(fileName)
    
    idCol = [c for c in df.columns if c.endswith("Id")]
    
    if not len(idCol):
        return
    
    df.drop(idCol[0], axis=1, inplace=True)
    export(df, fileName)
    print("Removed reference column from " + fileName)
    print("----------------------------------------------------------------------------------------------------------------------------")
    alteredFiles += 1


def tryRenameXYCols(fileName : str):
    global alteredFiles
    df = load(fileName)

    xyCols = [c for c in df.columns if c.endswith(("_x", "_y"))]

    if not len(xyCols):
        return
    
    for i in xyCols:
        if i.endswith("_x"):
            df.rename(columns={i: i.removesuffix("_x")}, inplace=True)
        else:
            df.rename(columns={i: "site" + i.removesuffix("_y").capitalize()}, inplace=True)
    
    print("Renamed x/y columns from " + fileName)
    print("----------------------------------------------------------------------------------------------------------------------------")
    alteredFiles += 1

In [ ]:
alteredFiles = 0

for fileName in csvFiles:
    tryRemovingReferenceCol(fileName)
    tryRenameXYCols(fileName)

print("Modified files: " + str(alteredFiles))